## lora_diffusion

- https://github.com/cloneofsimo/lora
- https://github.com/microsoft/LoRA

In [3]:
from diffusers import DiffusionPipeline

from lora_diffusion import inject_trainable_lora, extract_lora_ups_down, monkeypatch_or_replace_lora

model_id = "/home/yang/sda/github/fuzzydiffusion/output/ldm"

# load model and scheduler
pipeline = DiffusionPipeline.from_pretrained(model_id)

unet = pipeline.unet
unet_lora_params, train_names = inject_trainable_lora(unet) 

print(train_names)

Loading pipeline components...:   0%|          | 0/3 [00:00<?, ?it/s]

The config attributes {'timestep_values': None, 'timesteps': 1000} were passed to DDIMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


['to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0', 'to_q', 'to_k', 'to_v', '0']


In [1]:
from diffusers import DiffusionPipeline
from omegaconf import OmegaConf

import loralib as lora
from ldm.util import instantiate_from_config

def get_parameter_number(model):
    total_num = sum(p.numel() for p in model.parameters())
    trainable_num = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return {'Total': total_num, 'Trainable': trainable_num}

config = OmegaConf.load("/home/yang/sda/github/fuzzydiffusion/src/config/latent-diffusion/lsun_churches-ldm-kl-8.yaml")
model_config = config.pop("model", OmegaConf.create())

ldm = instantiate_from_config(model_config)
first_stage_model = ldm.first_stage_model
cond_stage_model = ldm.cond_stage_model
unet = ldm.model.diffusion_model
lora.mark_only_lora_as_trainable(unet)
trainable = []
frozen = []
model = ldm
for name, param in model.named_parameters():
    if param.requires_grad:
        trainable.append(name)
    else:
        frozen.append(name)
print(len(trainable))
print(len(frozen))
print(get_parameter_number(model))

2023-10-18 15:48:18.745076: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-18 15:48:18.768210: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-18 15:48:19.096910: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 308.75 M params.
Keeping EMAs of 690.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Training LatentDiffusion as an unconditional model.
Restored from /home/yang/sda/github/fuzzydiffusion/output/models/ldm/lsun_churches256/model.ckpt with 1065 missing and 565 unexpected keys
Missing Keys: ['model.diffusion_model.time_embed.0.lora_A', 'model.diffusion_model.time_embed.0.lora_B', 'model.diffusion_model.time_embed.2.lora_A', 'model.diffusion_model.time_embed.2.lora_B', 'model.diffusion_model.input_blocks.0.0.lora_A', 'model.diffusion_model.input_blocks.0.0.lora_B', 'model.diffusion_model.input_blocks.0.0.conv.weight', 'model.diffusion_model.input_blocks.0.0.conv.bias', 'model.diffusion_model.input_blocks.1.0.in_layers.2.lora_A', 'model.diffusion_model.input_blocks.1.0.in_layers.2.lora_B', 'model.diff